In [137]:
#execution requirements.txt
#! pip install -r requirements.txt

# Librairie pddl

### Exploration librairie

In [187]:
from pddl.parser.domain import DomainParser
from pddl.parser.problem import ProblemParser
import pddl
from itertools import product
import copy

#import data/domain.pddl in str

domain_str = open("data/domain.pddl").read()
problem_str = open("data/problem.pddl").read()  

# print(domain_str)
# print(problem_str)


domain = DomainParser()(domain_str)
problem = ProblemParser()(problem_str)

init_state = problem.init
goal_state = problem.goal

# PROBLEM
print("Initial : ", list(init_state))
print("Goal : ", problem.goal.__getstate__()['_operands'])
print("0bjects :", list(problem.objects))


# DOMAIN
print("Predicates : ", list(domain.predicates))
print("Actions : ", list(domain.actions))
print("Action 0 Name : ",list((domain.actions))[0].name)
print("Action 0 Precondition : ",list(list((domain.actions))[0].precondition.operands))
print("Action 0 Effect : ",list(list((domain.actions))[0].effect.operands))
print("Action 0 Parameters : ",list(list((domain.actions))[0].parameters))


Initial :  [Predicate(on, d2, d3), Predicate(smaller, d0, d3), Predicate(clear, d0), Predicate(clear, p2), Predicate(smaller, d2, p0), Predicate(smaller, d3, p0), Predicate(clear, p1), Predicate(smaller, d1, p0), Predicate(smaller, d0, p0), Predicate(on, d3, p0), Predicate(smaller, d1, d2), Predicate(smaller, d2, p2), Predicate(smaller, d1, p1), Predicate(smaller, d3, p2), Predicate(smaller, d1, p2), Predicate(on, d1, d2), Predicate(smaller, d2, p1), Predicate(smaller, d0, p2), Predicate(smaller, d3, p1), Predicate(smaller, d0, d1), Predicate(smaller, d1, d3), Predicate(smaller, d0, d2), Predicate(on, d0, d1), Predicate(smaller, d0, p1), Predicate(smaller, d2, d3)]
Goal :  [Predicate(on, d0, d1), Predicate(on, d1, d2), Predicate(on, d2, d3), Predicate(on, d3, p2)]
0bjects : [Constant(d3), Constant(p0), Constant(d2), Constant(d0), Constant(d1), Constant(p2), Constant(p1)]
Predicates :  [Predicate(on, ?x, ?y), Predicate(clear, ?x), Predicate(smaller, ?x, ?y)]
Actions :  [Action(move, par

### Fonction pour générer toutes les actions possibles du problème

In [186]:


def generate_klist(variables, k):
    return list(product(variables, repeat=k))

def all_actions(domain, problem):
    Actions = []
    # pour toute les actions
    for a in range(len(domain.actions)):
        action = list(domain.actions)[a]
        possiblity=generate_klist(list(problem.objects), len(list(action.parameters)))
        # pour toutes les combinaisons de variables possible pour une action
        for klist in possiblity:
            dict_action = {}
            # name
            dict_action['name'] = action.name.__str__()+"_"+'_'.join([v.__str__() for v in klist])
            dict_var=dict(zip(action.parameters, klist))
            # preconditions
            dict_action['preconditions'] = []

            for p in range(len(list(list((domain.actions))[a].precondition.operands))):
                precondition = copy.deepcopy(list(list((domain.actions))[a].precondition.operands)[:])[p]
                terms =tuple([dict_var[v] for v in precondition.__getstate__()['_terms']])
                precondition.__getstate__()['_terms']=terms
                dict_action['preconditions'].append(precondition)

            effects=copy.deepcopy(list(list((domain.actions))[a].effect.operands))

            # positive effects
            dict_action['positive effects'] = []

            # negative effects
            dict_action['negative effects'] = []

            for e in range(len(effects)):
                effect = effects[e]
                if type(effect) is pddl.logic.predicates.Predicate:
                    terms =tuple([dict_var[v] for v in effect.__getstate__()['_terms']])
                    effect.__getstate__()['_terms']=terms
                    dict_action['positive effects'].append(effect)  

                elif type(effect) is pddl.logic.base.Not:
                    terms =tuple([dict_var[v] for v in effect.__getstate__()['_arg'].__getstate__()['_terms']])
                    effect.__getstate__()['_arg'].__getstate__()['_terms']=terms
                    dict_action['negative effects'].append(effect)         
                          
            Actions.append(dict_action) 
    return Actions
            
# print(all_actions(domain, problem))

### Heuristique

### Fonction de vérification du planner

### Algo de planification

# Librairie unified-planning

### Solveur


In [ ]:
#! pip install 'unified-planning[fast-downward]'

In [188]:
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import *
from unified_planning.engines import PlanGenerationResultStatus

reader = PDDLReader()
pddl_problem = reader.parse_problem('data/domain.pddl', 'data/problem.pddl')
# print(pddl_problem)


with OneshotPlanner(
    problem_kind=pddl_problem.kind,
    optimality_guarantee=PlanGenerationResultStatus.SOLVED_OPTIMALLY,
) as planner:
    final_report = planner.solve(pddl_problem)
    plan = final_report.plan

print("Plan found")   
print(plan)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 10 of `/var/folders/t8/bqm3crl96qx1bmc4ty9hg7_w0000gn/T/ipykernel_63708/4087593541.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

Plan found
SequentialPlan:
    move(d0, d1, p1)
    move(d1, d2, p2)
    move(d0, p1, d1)
    move(d2, d3, p1)
    move(d0, d1, d3)
    move(d1, p2, d2)
    move(d0, d3, d1)
    move(d3, p0, p2)
    move(d0, d1, d3)
    move(d1, d2, p0)
    move(d0, d3, d1)
    move(d2, p1, d3)
    move(d0, d1, p1)
    move(d1, p0, d2)
    move(d0, p1, d1)


### Actions applicables, paramètres applicables modification de l'état

In [ ]:
# print(pddl_problem)
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import *
from unified_planning.plans import ActionInstance

reader = PDDLReader()
pddl_problem = reader.parse_problem('data/domain.pddl', 'data/problem.pddl')

simulator = SequentialSimulator(pddl_problem)
init_state=simulator.get_initial_state()
goal=pddl_problem.goals

(action_name, para) = tuple(simulator.get_applicable_actions(init_state))[0] # récup de la première fonction applicable et des paramètre qui conviennent

action_instance = ActionInstance(action_name, para) #création de l'objet pour appliquer la fonction

new_state = simulator.apply(init_state, action_instance) #on applique la fonction sur l'état

print(new_state)


{clear(d1): true, on(d0, p1): true, on(d0, d1): false, clear(p1): false, clear(d0): true, clear(p2): true, on(d1, d2): true, on(d2, d3): true, on(d3, p0): true, smaller(d0, d1): true, smaller(d0, d2): true, smaller(d0, d3): true, smaller(d0, p0): true, smaller(d0, p1): true, smaller(d0, p2): true, smaller(d1, d2): true, smaller(d1, d3): true, smaller(d1, p0): true, smaller(d1, p1): true, smaller(d1, p2): true, smaller(d2, d3): true, smaller(d2, p0): true, smaller(d2, p1): true, smaller(d2, p2): true, smaller(d3, p0): true, smaller(d3, p1): true, smaller(d3, p2): true, clear(d2): false, clear(d3): false, clear(p0): false, on(d0, d0): false, on(d1, d0): false, on(d2, d0): false, on(d3, d0): false, on(p0, d0): false, on(p1, d0): false, on(p2, d0): false, on(d1, d1): false, on(d2, d1): false, on(d3, d1): false, on(p0, d1): false, on(p1, d1): false, on(p2, d1): false, on(d0, d2): false, on(d2, d2): false, on(d3, d2): false, on(p0, d2): false, on(p1, d2): false, on(p2, d2): false, on(d0, d3)

# Librairie pddlpy

In [ ]:
import pddlpy

# Load your domain and problem PDDL files
domprob = pddlpy.DomainProblem('data/domain.pddl', 'data/problem.pddl')
print(domprob.worldobjects())
print(domprob.initialstate())
print(domprob.goals())
print(domprob.operators())
print(domprob.ground_operator('move'))
print(domprob.domain.operators['move'])

help(domprob.ground_operator('move'))